# Parallel Processing with Dask
### by [Richard W. Evans](https://sites.google.com/site/rickecon/), May 2020
The code in this Jupyter notebook was written using Python 3.7.

Parallel processing and high performance computing have become essential tools with the increase in the size and complexity of datasets and with the advances in dimensionality and solution algorithms of mathematical models. Further, simulation methods and statistical learning training methods benefit from the ability to scale computations on distributed systems like Google Cloud Services (GCS), Amazon Web Services (AWS), and Microsoft Azure. Jon Clindaniel is teaching an excellent class this term (MACS 30123, Large Scale Computing for the Social Sciences) that presents a much deeper dive into the subject matter. 

The standard laptop has multiple processors and usually comes equipped with a graphics card with hundreds of graphics processing units (GPUs). Each student's laptop can be transformed into a distributed computing system or, in other words, a supercomputer. Further, University of Chicago students have access to the [Midway cluster](https://rcc.uchicago.edu/resources/high-performance-computing) hosted by the [Research Computing Center (RCC)](https://rcc.uchicago.edu/). And finally, University of Chicago students with very sophisticated and high profile projects can apply for allocations on the [Argonne National Laboratory](https://www.anl.gov/) supercomputer, which has the [22nd and 28th most powerful](https://www.top500.org/list/2019/11/) clusters in the world as of November 2019.

We will focus on [Dask](https://dask.org/), which is the primary parallel processing library for Python. High performance computing has traditionally been executed on supercomputers through low-level languages such as C++ and Fortran. But most major supercomputers now support Python. Furthermore, Python has become the primary supervising architecture language at many of the national laboratories because it is quicker to write (developer time is more expensive than computer time) and because Python can serve as a wrapper for other low-level languages.

In addition to Python being a supported language on most supercomputers, it is also a language with sophisticated parallel processing ability on whatever system you use personally--laptop, desktop, or high performance server.

A great resource for learning high performance computing with Dask is the [Dask tutorial](https://github.com/dask/dask-tutorial) on GitHub. Another great resource is the Open Source Economics Laboratory (OSE Lab) [HPC training](https://github.com/sischei/OSE2019) by [Simon Scheidegger](https://sites.google.com/site/simonscheidegger).

## 1. Reasons we need HPC

* Large data must be stored across multiple hard drives
* Large data I/O must be optimized
* Many technical computations can be parallelized
* Simulation methods can be parallelized
* Sampling methods can be parallelized

Previous thinking was that we could just wait for processors and memory to become powerful enough, compact enough, and efficient enough to accomodate our increasing data sizes and methods complexity. However, the steady-increases in processor efficiency began to plateau in the mid 2000s. The March 12, 2016 issue of *The Economist* had an article entitled "After Moore's Law" that provided the following graphic of the trailing off of individual processor power.

![Processor Plateau Fig](images/ProcessorPlateauFig.png "Processor Plateau Fig")

Maximum clock speed has slowed despite the steady (exponential) increase in transistors per chip. The reasons for this slowdown are the following.

* Hitting heat limits, cannot dissipate processing heat fast enough
* Current leakage
* Power consumption too high
* Memory requirements too high

But note that the price per transistor has also plateaued. The solution to this problem has been more parallel architecture and distributed systems as well as more specific architecture tailored to specific problems.

## 2. Serial versus Parallel
Serial computation is a set of instructions for which the order of computational instructions is important. Serial instruction tasks cannot be parallelized. Examples of computational tasks that must be serial are recursions and sorting.

![Serial Process](images/SerialProcess.png "Serial Process")

A parallel process breaks a problem up into pieces, executes the instructions associated with each piece, then brings the answers back together. Examples of computational tasks that can be parallelized are simulation, bootstrapping, some for-loops, pieces of numerical methods.

![Parallel Process](images/ParallelProcess.png "Parallel Process")

## 3. Concurrency processing paradigms
[Real Python](https://realpython.com/) has a great free tutorial on concurrency in Python ("[Speed Up Your Python Program With Concurrency](https://realpython.com/python-concurrency/)"). Concurrency is more general than parallel processing. Concurrency means a set of tasks are given to a processor or set of processors at the same time. From that point, there are a number of ways the CPUs can approach handling those tasks at the same time. I like the division of  paradigms at the beginning of the Real Python concurrency tutorial.

| Concurrency Type | Switching Decision | Number of Processors |
| :--- | :--- | :---: |
| Pre-emptive multitasking (`threading`) | The operating system decides when to switch tasks external to Python | 1 |
| Cooperative multitasking (`asyncio`) | The tasks decide when to give up control | 1 |
| Multiprocessing (`multiprocessing`) | The processes all run at the same time on different processors | Many |

When we think of high performance computing in economics, we usually think of broadcasting processes on to many different cores to be run independently at the same time--the multiprocessing paradigm. However, the two multitasking paradigms (`threading` and `asyncio`) are extremely valuable for input/output intensive applications that have to wait for server responses.

This notebook will leave the multitasking paradigms for you to learn on your own. Real Python has some great tutorials. We will focus here on multiprocessing. However, the `Dask` library in Python incorporates all three paradigms in its functionality and command structure.

## 4. Low-level to high-level parallel/multiprocessing programming paradigms
[Message passing interface (MPI)](https://en.wikipedia.org/wiki/Message_Passing_Interface) and OpenMP (open multi-processing) are the two main paradigms for executing high performance computing and parallel operations. These platforms are built on C, C++, and Fortran code, and are difficult to learn and to program. Most modern computer languages have bindings for these two platforms that allow users to access the functionality of MPI and OpenMP from the native environment of their high-level language of choice (e.g., Python, R, Java).

Python has a particularly powerful library for parallel data I/O and parallel computation called [Dask](https://dask.pydata.org/en/latest/). A large number of great webcast tutorials and Jupyter notebooks for using Dask are available on the web. A particularly valuable [webcast Dask tutorial](https://www.youtube.com/watch?v=mbfsog3e5DA) came from the SciPy 2017 meeting in Austin Texas. Dask has a great GitHub repository of tutorials [https://github.com/dask/dask-tutorial](https://github.com/dask/dask-tutorial). And the core maintainer of Dask, Matthew Rocklin, has a great YouTube channel with a [Dask playlist](https://www.youtube.com/playlist?list=PLRtz5iA93T4PQvWuoMnIyEIz1fXiJ5Pri) of tutorial videos.

The following is a summary of the 2017 video. However, I recommend the updated videos in the [Dask playlist](https://www.youtube.com/playlist?list=PLRtz5iA93T4PQvWuoMnIyEIz1fXiJ5Pri) on the Matthew Rocklin YouTube channel.

* (0:00-21:48) James Crist: Using Dask delayed
* (21:53-56:59) Skipper Seabold: Dask DataFrames
  * Dask DataFrames do not handle multiple pandas indices. Just choose the one that is the most intensive, and then choose subroutines to use any secondary indices
  * Don't distribute stuff that will fit in memory. Don't add more complexity when not needed
* (57:00-1:34:03) Martin Durant: Dask arrays (NumPy)
  * Divides arrays into chunks that vary across all dimensions
  * Dask DataFrames divides Pandas DataFrames into separate similar DataFrame chunks.
  * Uses hdf5 data
  * You want each piece of memory to be multiple megabytes but less than a gigabyte
* (1:34:05-1:49:30) Skipper Seabold: Bag (parallel lists for semi-structured data)
  * Good place to clean large data, messy data, nested data, transactional data
  * Use foldby instead of groupby with bags
* (1:49:30-2:06:28) James Crist: Schedulers
  * Multiprocessing scheduler has some flaws
  * Distributed scheduler fixes this
  * Dask functionality with GPU's is a little limited
* (2:06:37-2:20:58) Martin Durant: Distributed efficiency and DataFrames
  * On distributed systems, want to store separate pandas dataframes in memory on multiple machines.
  * Use the .persist method
* (2:21:04-2:30:33) James Crist: Advanced distributed features
  * Has functions like gather for asyncronous computations
* (2:31:00-2:37:14) James Crist: History and progress of Dask, final Q&A

## 5. Using Dask delayed
For the remaining sections, we will use tutorial notebooks from the Dask GitHub tutorial repository ([https://github.com/dask/dask-tutorial](https://github.com/dask/dask-tutorial)). You should clone or fork this repository so you can work through the listed notebooks on your local machine.

Use Dask tutorial notebook [01_dask.delayed.ipynb](https://github.com/dask/dask-tutorial/blob/master/01_dask.delayed.ipynb). You will need to follow the instructions in the [README.md](https://github.com/dask/dask-tutorial/blob/master/README.md) on the [Dask tutorial page](https://github.com/dask/dask-tutorial) in order to be able to access the corresponding datasets and execute the cell blocks. Note that the `python prep.py` step takes a few minutes because it is creating a bunch of large data sets.

Also, go through the first part of the [01x_lazy.ipynb](https://github.com/dask/dask-tutorial/blob/master/01x_lazy.ipynb) notebook that covers the `@delayed` decorator.

Some notes.

* We use the `time.sleep()` function to make simple functions take more time. This is trivial way to introduce functions that have measurable (> 1 second) computation time.
* The `dask.delayed` decorator stages operations that are to be parallelized. Any function `dask.delayed` touches becomes lazy and runs later. Any data touched by `dask.delayed` will make any instruction that calls the data be delayed or lazy and run later.
* `dask.compute()` runs the delayed objects.

 ## 6. Using Dask arrays (NumPy)
 Use Dask tutorial notebook [03_array.ipynb](https://github.com/dask/dask-tutorial/blob/master/03_array.ipynb). This tutorial uses the HDF5 data format. [HDF5](https://www.hdfgroup.org/solutions/hdf5/) stands for hierarchical data format, and is a popular and efficient format in which to store large *n*-dimensional data or *n*-dimensional arrays. HDF5 files have a nice interface with both native python (`h5py` library) and through `Dask`.

For working with HDF5 files in `Python`, you can go to the [`h5py` users manual](http://docs.h5py.org/en/latest/) or [FAQ](http://docs.h5py.org/en/latest/faq.html) page. Or you can go to the nice O'Reilley book by Collette (2013).

Go through first example, up to complex graph GIF. Then students can go through other examples on own. Make sure to cover the "Limitations" section at the end of the notebook.

## 7. Using Dask DataFrames (Pandas)
Use Dask tutorial notebook [04_dataframe.ipynb](https://github.com/dask/dask-tutorial/blob/master/04_dataframe.ipynb). This Dask functionality allows you to easily implement parallel I/O for datasets that are larger than RAM, larger than hard disk, or just large. This is faster than Pandas for most main functions, but works like Pandas. Make sure to go through the limitations section at the end.

## 8. More complicated parallelism
You can imagine much more complicated graphs of parallel processes. `Dask` allows you to perform more customized computation of parallel graphs in which you control what processes get sent to which workers.

Use `Dask` tutorial notebook [05_distributed.ipynb](https://github.com/dask/dask-tutorial/blob/master/05_distributed.ipynb) as an introduction to the more advanced `dask.distributed` scheduler. The [06_distributed_advanced.ipynb](https://github.com/dask/dask-tutorial/blob/master/06_distributed_advanced.ipynb) notebook is more advanced and requires some understanding of MPI. A good prerequisite is a tutorial on Python's `multiprocessing` library, which is a wrapper for MPI. Also good is the `mpi4py` library.

### 8.1. Example
Lets do a serial example and a parallel example in which we draw 10 datasets of 1 million elements each from a normal distribution $N(\mu, \sigma)$. From each of these datasets, we will compute the mean, standard deviation, and 90th percentile. In serial that is done by the following code.

In [ ]:
import numpy as np
import scipy.stats as sts
import timeit

Sims = 12
ObsPerSim = 1000000
mu_norm = 10
std_norm = 1.8
seed_vec = np.arange(1, Sims + 1)

start_time = timeit.default_timer()

results_ser = np.zeros([Sims, 3])

for sim in range(Sims):
    data = sts.lognorm.rvs(s=std_norm, scale=np.exp(mu_norm),
                           size=ObsPerSim, random_state=seed_vec[sim])
    results_ser[sim, 0] = data.mean()
    results_ser[sim, 1] = data.std()
    results_ser[sim, 2] = sorted(data)[900000]
    
elapsed_time = timeit.default_timer() - start_time
print('Elapsed time=', elapsed_time, 'seconds')
    
print(results_ser)

In parallel, we can do that with the following code. We write a function that takes that parameters necessary for each of the 10 simulations, we start up the distributed scheduluer, we define the function as a `Dask.delayed` object, then we compute it using `Dask.compute`.

In [ ]:
# First let's check how many cores we have available
import multiprocessing

num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

In [ ]:
from dask import compute, delayed
import dask.multiprocessing
from pprint import pprint

start_time = timeit.default_timer()

def sim_stats(sim_val, std, mu, obs, seed):
    print('Working on simulation', sim_val)
    data = sts.lognorm.rvs(s=std, scale=np.exp(mu), size=obs,
                           random_state=seed)
    results_mean = data.mean()
    results_std = data.std()
    results_90pct = sorted(data)[int(0.9 * obs)]
    results = np.array([results_mean, results_std, results_90pct])
    
    return results

lazy_values = []
for sim in range(Sims):
    lazy_values.append(delayed(sim_stats)(sim, std_norm, mu_norm, ObsPerSim, seed_vec[sim]))

results_par = compute(*lazy_values, scheduler=dask.multiprocessing.get, num_workers=num_cores)

elapsed_time = timeit.default_timer() - start_time
print('Elapsed time=', elapsed_time, 'seconds')

pprint(results_par)

## 9. Things to watch out for
There are some common problems that arise when using Dask or any other parallel computing strategy.

### 9.1. Memory per worker/node/core
You might get an error that says something like the following:
```
2445distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.15 GB -- Worker memory limit: 4.18 GB
```
or the following:
```
distributed.worker -- WARNING -- Worker exceeded 95% memory budget. restarting
```
When you distribute the computing of processes, each worker or node or core has a memory allocation. The total amount is the RAM in your computer. Dask has some built in limits, and 60% of the memory per worker is a good target (see this [Dask documentation link](https://distributed.dask.org/en/latest/worker.html#memory-management)).
```
distributed:
    worker:
        # Fractions of worker memory at which we take action to avoid memory blowup
        # Set any of the lower three values to False to turn off the behavior entirely
        memory:
          target: 0.60  # target fraction to stay below
          spill: 0.70  # fraction at which we spill to disk
          pause: 0.80  # fraction at which we pause worker threads
          terminate: 0.95  # fraction at which we terminate the worker
```
A good rule of thumb is to divide your machine's RAM allocation by the number of workers you are using. Then make sure that no worker receives more than 60 percent of that allocation per worker. This type of memory management is a key skill in any high performance computing implementation (C++, Fortran, Python, etc.)

### 9.2. Smart/flexible scaling on a cluster

## 10. References
* Collette, Andrew, *Python and HDF5*, O'Reilly (2013).
* *The Economist*, "[After Moore's Law](https://www.economist.com/technology-quarterly/2016-03-12/after-moores-law)", *The Economist*, (March 12, 2016).